# Redes neuronales

Se utilizó el dataset <b>Fasion MNIST</b> de https://www.kaggle.com/zalando-research/fashionmnist

## Import de librerías

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
from scipy import optimize as op

In [2]:
from RedesNeuronales import *
from utilities import *

In [3]:
import time

## Lectura de CSV

In [4]:
dataset = pd.read_csv('fashion-mnist_train.csv')
datasetTest = pd.read_csv('fashion-mnist_test.csv')

In [5]:
dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
datasetTest.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data = dataset.iloc[:, :].values
train, _, _ = getPartitions(data, 1, 0)

In [8]:
dataTest = datasetTest.iloc[:, :].values
test, _, _ = getPartitions(dataTest, 1, 0)

## Datos de entrenamiento y cross validation

In [9]:
y_train = train[:, 0].reshape(-1, 1)
x_train = train[:, 1:]

In [10]:
y_test = test[:, 0].reshape(-1, 1)
x_test = test[:, 1:].reshape(-1, 784)

In [11]:
NEURAL_NET = np.array([
    x_test.shape[1],
    125,
    10
])

theta_shapes = np.hstack((
    NEURAL_NET[1:].reshape(len(NEURAL_NET) - 1, 1),
    (NEURAL_NET[:-1] + 1).reshape(len(NEURAL_NET) - 1, 1)
))

In [12]:
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

X = x_train / 1000.0
Y = (y_train == np.array(range(10))).astype(int)

In [18]:
flat_thetas = readFile("flat_thetas.txt")

In [19]:
Thetas = inflate_matrixes(flat_thetas, theta_shapes)

In [22]:
a = feed_forward(Thetas, x_test)

In [23]:
success = 0
for i in range(len(a[-1])):
    value = np.where(np.isclose(a[-1][i], max(a[-1][i])))[0][0]
    
    if (value == y_test[i][0]):
        success += 1
        
print("Success: ", success, success * 100/len(a[-1]))

Success:  4758 47.58


In [65]:
print(len(a[1]))

60000


In [19]:
m, layers = len(X), len(theta_shapes) + 1
thetas = inflate_matrixes(flat_thetas, theta_shapes)
a = feed_forward(thetas, X) # 2.2
deltas = [*range(layers - 1), a[-1] - Y]

In [84]:
flatten_list_of_arrays(
    Deltas
)

array([3.84562685e-04, 8.77783006e-12, 4.11573873e-10, ...,
       9.00226084e-01, 9.00205780e-01, 9.00216149e-01])

In [35]:
for i in range(layers - 2, 0, -1):
    deltas[i] = (deltas[i + 1] @ np.delete(thetas[i], 0, 1)) * (a[i] * (1 - a[i]))

In [87]:
start_time = time.time()
Deltas = []
for i in range(layers - 1):
    Deltas.append(
        (deltas[i + 1].T
        @
        np.hstack((
            np.ones(len(a[i])).reshape(len(a[i]), 1),
            a[i]
        ))) / m
    )
    
flat = flatten_list_of_arrays(
    Deltas
)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.3350691795349121 seconds ---


In [88]:
start_time = time.time()
arregloDeltas = []
for n in range(layers - 1):
    f1, c1 = thetas[n].shape
    deltaMayuscula = np.zeros(f1*c1).reshape(f1,c1) 
    f2, _ = a[n].shape
    for k in range(f2): 
        f3 = len(deltas[n + 1][k])
        f4 = len(a[n][k])
        deltaMayuscula = deltaMayuscula + ((deltas[n + 1][k]).reshape(f3,1) @ ((np.append([1],a[n][k])).reshape(f4 + 1,1)).T)

    arregloDeltas.append(deltaMayuscula) 

arregloDeltas = np.asarray(arregloDeltas)

# Paso 3
flatD = flatten_list_of_arrays([
    np.true_divide(arreglo, m)
    for arreglo in arregloDeltas
])
print("--- %s seconds ---" % (time.time() - start_time))

--- 12.513553380966187 seconds ---


In [89]:
print(flat)

[3.84562685e-04 8.77783006e-12 4.11573873e-10 ... 9.00226084e-01
 9.00205780e-01 9.00216149e-01]


In [90]:
print(flatD)

[3.84562685e-04 8.77783006e-12 4.11573873e-10 ... 9.00226084e-01
 9.00205780e-01 9.00216149e-01]


In [12]:
flat_thetas = readFile("text.txt")

In [13]:
Thetas = inflate_matrixes(flat_thetas, theta_shapes)

In [14]:
a = feed_forward(Thetas, x_test)

In [30]:
print(len(a[2]),len(a[2][0]))

60000 10


In [12]:
success = 0
for i in range(len(a[-1])):
    value = np.where(np.isclose(a[-1][i], max(a[-1][i])))[0][0]
    
    if (value == y_test[i][0]):
        success += 1
        
print("Success: ", success, success * 100/len(a[-1]))

Success:  6030 10.05
